In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.float_format', lambda x: '{:.0f}'.format(x))

from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

import time

In [2]:
train_df = pd.read_parquet("../data/raw/train_data.pqt")
test_df = pd.read_parquet("../data/raw/test_data.pqt")


cat_cols = [
    "channel_code", "city", "city_type",
    "okved", "segment", "start_cluster",
    "index_city_code", "ogrn_month", "ogrn_year",
]


train_df[cat_cols] = train_df[cat_cols].astype("category")
test_df[cat_cols] = test_df[cat_cols].astype("category")

### Объединение датасета 

In [3]:
df = pd.concat([train_df, test_df])

In [4]:
df

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster
0,0,month_1,1,1,1,1,channel_code_5,city_23,city_type_0,index_city_code_39,...,1,1,1,1,1,1,0,1,"{α, γ}",{other}
1,0,month_2,1,1,2,1,channel_code_5,city_23,city_type_0,index_city_code_39,...,1,0,1,1,1,1,0,1,"{α, γ}",{other}
2,0,month_3,1,1,0,1,channel_code_5,city_23,city_type_0,index_city_code_39,...,1,0,1,1,1,1,0,1,"{α, γ}",{other}
3,1,month_1,-0,-0,-0,-0,channel_code_2,city_14,city_type_0,NaN,...,1,0,0,1,1,-0,0,0,{other},{other}
4,1,month_2,-0,-0,-0,-0,channel_code_2,city_14,city_type_0,NaN,...,1,0,0,1,1,-0,0,0,{other},{other}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290115,299998,month_5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0,NaN,NaN,-0,NaN,NaN,{},NaN
290116,299998,month_6,-0,-0,-0,-0,channel_code_9,city_25,city_type_0,index_city_code_30,...,1,0,-0,1,0,-0,0,0,NaN,NaN
290117,299999,month_4,-0,-0,-0,-0,channel_code_9,city_6,city_type_0,index_city_code_34,...,1,0,-0,1,1,-0,0,0,{α},NaN
290118,299999,month_5,-0,-0,-0,-0,channel_code_9,city_6,city_type_0,index_city_code_34,...,1,0,-0,1,1,-0,0,0,{α},NaN


### Преобразование типов

In [5]:
# int32 от -2,147,483,648 до 2,147,483,647
# float16 от -65504 до 65504
df['id'] = df['id'].astype('int32')

float64_columns = df.select_dtypes(include=['float64']).columns
df[float64_columns] = df[float64_columns].astype('float16')

### Разеделние датасета

In [6]:
train_df = df.iloc[:len(train_df)]
test_df = df.iloc[len(train_df):]

### Сохранение датасета

In [7]:
train_df = df.iloc[:len(train_df)]
test_df = df.iloc[len(train_df):]